In [3]:
import os
os.getcwd()  # Check the current working directory

# # Step 3: Add the parent directory to the sys.path
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Step 4: Import the class from the module in the dipy directory
import dipy.dipy as di


In [4]:
# For performance testing:
import time

#
# Example usage: digital asset or nothing
#

s0_value = 100.0
K_value = 110.0
r_value = 0.05
sigma_value = 0.3
dt_value = 1
u_value = 0.5 # scipy.stats.uniform.rvs()
z_value = 0.5

N = 1000000
seed = 1




###
### 1. Define inputs used in di.
###

# Computation settings
di.setBackend('torch')

s0 = di.variable(s0_value, 'input','s0')
K = di.variable(K_value, 'input','B')
r = di.variable(r_value, 'input','r')
sigma = di.variable(sigma_value, 'input','sigma')
dt = di.variable(dt_value, 'constant','dt')
u = di.variable(u_value, 'randomVariable','u')
z = di.variable(z_value, 'randomVariableNormal','z')

###
### 2. Record Tape: Standard Monte Carlo
###

# Test phase
#di.seed(seed)

times = []
test_iterations = 100
for _ in range(test_iterations):
    # Start time
    tic = time.time()

    # I'd want to have z = di.normal(....), it should just call z.NewSample(N)...
    z.NewSample(N) # torch.normal(mean=0, std=1, size=(1,sampleSize))

    s = s0 * di.exp((r - sigma **2 / 2) * dt + sigma * di.sqrt(dt) * z)
    payoff =  di.if_(s > K, s - K, 0)

    PV_standard = di.exp(-r * dt) * di.sum(payoff) / N

    ## Result and Delta
    result_standard = PV_standard.eval()
    delta_standard = PV_standard.grad(s0)

    #Stop time
    toc = time.time()
    spent = toc - tic
    times.append(spent)

mean_time = sum(times) / test_iterations
print('spent: ', mean_time)
print('Standard MC result:', result_standard)
print('Delta approximation:', delta_standard)


## Performance Studies
print(f"Performance test of standard: ")
PV_standard.DoPerformanceTest(s0)

spent:  0.0073398232460021974
Standard MC result: 10.005160331726074
Delta approximation: 0.4997560977935791
Performance test of standard: 
Starting performance test:
Mean execution time: 0.001465 seconds
Variance in execution time: 0.000016 seconds
